In [1]:

import os
import chromadb

client = chromadb.CloudClient(
  api_key=os.getenv('CHROMA_API_KEY'),
  tenant=os.getenv('CHROMA_TENANT'),
  database=os.getenv('CHROMA_DATABASE')
)

In [2]:
specimens = client.get_collection('specimens')

In [4]:
from openai import AzureOpenAI
import os

azure_client = AzureOpenAI(
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT')
)

response = azure_client.embeddings.create(
    input="A brave new world",
    model="text-embedding-3-large",
    dimensions=1024
)

In [16]:
embeddings = response.data[0].embedding

In [19]:
results = specimens.query(query_embeddings=embeddings)

In [25]:
[metadata['specimen_name'] for metadata in results['metadatas'][0][:5]]

['RM 2352 falcon statuette',
 'RM 7295 dog skull',
 'RM 2019-04-07 frozen hawk',
 'RM 5092 hippo skull',
 'RM 2392 baboon skull']